In [2]:
import numpy as np
import pandas as pd
import scipy.linalg
from numpy.linalg import matrix_power
from pyquil.noise import damping_after_dephasing
from qiskit import *

In [3]:
def dm_to_bloch_reg(rho):
    # rho is a density matrix
    state = dm_to_bloch_vector(rho)
    state = cartesian_to_spherical(state)

    # state is now (theta, phi, r)
    for i, intv in enumerate(thetas):
        if (state[0] in intv):
            theta_reg = i
    for i, intv in enumerate(phis):
        if (state[1] in intv):
            phi_reg = i
    for i, intv in enumerate(radii):
        if (state[2] in intv):
            r_reg = i

    if (theta_reg == 0):
        theta_reg = phi_reg = 0
    if (theta_reg == len(thetas)-1):
        theta_reg = len(thetas)-1
        phi_reg = len(phis)-1

    return (theta_reg, phi_reg, r_reg)

def dm_to_bloch_vector(rho):
    x = np.array([[0,1],[1,0]])
    y = np.array([[0,-1j],[1j,0]])
    z = np.array([[1,0],[0,-1]])

    # where rho is a density matrix
    return (np.trace(rho @ x), np.trace(rho @ y), np.trace(rho @ z))

def dm_to_polar_coords(rho):
    # rho is a density matrix
    return cartesian_to_spherical(dm_to_bloch_vector(rho))

def cartesian_to_spherical(state):
    x = state[0]
    y = state[1]
    z = state[2]

    r = np.sqrt(x**2 + y**2 + z**2)
    theta = max(0, np.arccos(z/r))
    phi = np.arctan2(y.real, x.real)
    if (phi < 0): phi += 2*np.pi

    return (theta, phi, r)

def spherical_to_cartesian(state):
    theta = state[0]
    phi = state[1]
    r = state[2]

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    if abs(x) < 0.0001:
        x = 0
    if abs(y) < 0.0001:
        y = 0
    if abs(z) < 0.0001:
        z = 0
    return (x, y, z)

def random_state_in_reg(reg):
    # where reg is a tuple specifying (theta, phi, radius)
    # returns a density matrix
    theta = np.random.uniform(thetas[reg[0]].left, thetas[reg[0]].right)

    # maybe consider the poles as one state
    if (reg[0] == 0 or reg == len(thetas)-1):
        phi = np.random.uniform(0, 2*np.pi)
    else:
        phi = np.random.uniform(phis[reg[1]].left, phis[reg[1]].right)
    r = np.random.uniform(radii[reg[2]].left, radii[reg[2]].right)

    state = spherical_to_cartesian((theta, phi, r))
    rho = (np.eye(2) + state[0]*np.array([[0,1],[1,0]]) + state[1]*np.array([[0, -1j], [1j, 0]]) + state[2]*np.array([[1,0], [0,-1]]))/2
    return np.matrix(rho)

def generate_target_state(n):
    s = np.matrix([1, 0])
    rho = np.outer(s, s.H)
    
    ht = GATES[0] @ GATES[1]
    rho = matrix_power(ht, n) @ rho @ matrix_power(ht.H, n)
        
    return rho

def apply_operator(rho, op): # add noise
    n_rho = np.matrix([[0, 0], [0, 0]])
    for k in NOISE[3]:
        n_rho = n_rho + np.matrix(op @ k @ rho @ np.matrix(k).H @ op.H)
    # return np.matrix(op @ rho @ op.H)
    return n_rho

def dm_fidelity(rho, sigma):
    rho_sqrt = scipy.linalg.sqrtm(rho)
    return np.trace(scipy.linalg.sqrtm(rho_sqrt @ sigma @ rho_sqrt))**2

In [249]:
damping_after_dephasing(5e-6, 5e-6, 5e-7)

[array([[0.98773211, 0.        ],
        [0.        , 0.93955984]]),
 array([[ 0.15615789,  0.        ],
        [ 0.        , -0.14854198]]),
 array([[0.        , 0.30469988],
        [0.        , 0.        ]]),
 array([[ 0.        , -0.04817226],
        [ 0.        ,  0.        ]])]

In [181]:
damping_after_dephasing(1e-6, 1e-6, 2e-7)

[array([[0.97591942, 0.        ],
        [0.        , 0.88304841]]),
 array([[ 0.21813136,  0.        ],
        [ 0.        , -0.19737342]]),
 array([[0.        , 0.41550478],
        [0.        , 0.        ]]),
 array([[ 0.        , -0.09287101],
        [ 0.        ,  0.        ]])]

In [246]:
damping_after_dephasing(2e-6, 2e-6, 2e-7)

[array([[0.98773211, 0.        ],
        [0.        , 0.93955984]]),
 array([[ 0.15615789,  0.        ],
        [ 0.        , -0.14854198]]),
 array([[0.        , 0.30469988],
        [0.        , 0.        ]]),
 array([[ 0.        , -0.04817226],
        [ 0.        ,  0.        ]])]

In [221]:
n = 10**2
k = 16

GATES = [
    np.matrix([[1, 1], [1, -1]]) / np.sqrt(2), # H
    np.matrix([[1, 0], [0, np.exp(1j * np.pi / 4)]]), # T
    np.matrix([[1, 0], [0, 1]]) # I
]

p = 0.01
t1 = 1e-6
t2 = 1e-6
gate_time = 2e-7
NOISE = [
    [np.matrix([[1, 0], [0, 1]])],
    [(1 - p) * np.matrix([[1, 0], [0, 1]]), p/3 * np.matrix([[0, 1], [1, 0]]), p/3 * np.matrix([[0, -1j], [1j, 0]]), p/3 * np.matrix([[1, 0], [0, -1]])],
    [(1 - p) * np.matrix([[1, 0], [0, 1]]), p * np.matrix([[1, 0], [0, -1]])],
    damping_after_dephasing(t1, t2, gate_time)
]

goal = generate_target_state(n=n)
thetas = np.array(pd.cut(np.linspace(0, np.pi, k), k, precision=10, include_lowest=True))
thetas[0] = pd.Interval(0, thetas[0].right, closed='both')
phis = np.array(pd.cut(np.linspace(0, 2*np.pi, 2*k), 2*k,  precision=10, include_lowest=True))
phis[0] = pd.Interval(0, phis[0].right, closed='both')
radii = pd.cut(np.linspace(0, 1, k), k, precision=10, include_lowest=True)

goal_reg = dm_to_bloch_reg(goal)

states = [(i, j, k) for i in range(len(thetas)) for j in range(len(phis)) for k in range(len(radii))]
values = np.zeros(len(thetas) * len(phis) * len(radii))

In [13]:
print(np.trace(goal@goal.H))
print(goal_reg)
# print(dm_to_bloch_vector(goal))
print(dm_to_polar_coords(goal))
print(goal)
print('0: ', np.trace(np.outer(np.matrix([1,0]), np.matrix([1,0]).H) @ goal))
print('1: ', np.trace(np.outer(np.matrix([0,1]), np.matrix([0,1]).H) @ goal))

(0.9999999967824147+0j)
(6, 25, 15)
((1.2735465608178806+3.0247676623947504e-17j), 5.071889081050288, (0.9999999983912073-9.078430084626834e-17j))
[[0.64644585-5.55111512e-17j 0.16818904+4.47510959e-01j]
 [0.16818904-4.47510959e-01j 0.35355415+0.00000000e+00j]]
0:  (0.6464458456762312-5.551115123125783e-17j)
1:  (0.35355415271497614+0j)


In [15]:
transitions = [np.zeros((len(states), len(states)), dtype=np.half) for i in range(len(GATES))]

# building transition matrices
for ind, s in enumerate(states):
    if (ind % 500 == 0): print('.', end='')
    for i in range(100):
        state = random_state_in_reg(s)
        state_ind = states.index(dm_to_bloch_reg(state))
        for j in range(len(GATES)):
            n_state = apply_operator(state, GATES[j])
            n_state_ind = states.index(dm_to_bloch_reg(n_state))
            transitions[j][state_ind][n_state_ind] += 1

for i in range(len(GATES)):
    for j in range(len(states)):
        transitions[i][j] = np.nan_to_num(transitions[i][j] / sum(transitions[i][j]))

.

C:\Users\nfber\anaconda3\envs\qc\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in cdouble_scalars


................

C:\Users\nfber\anaconda3\envs\qc\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [17]:
def R(state, action):
    # if (state == goal_reg):
    #     return 1
    #     if (action == 0 or action == 1):
    #         return 0.9
    #     else:
    #         return 1 # to encourage using identity
    # else:
    #     return 0

    if (state[0] == goal_reg[0] and state[1] == goal_reg[1]):
        return state[2]/k # pretty much the purity of the state
    else:
        return 0

In [18]:
def policy_eval(policy, discount_factor=0.8, epsilon=0.0001):
    V_old = np.zeros(len(states))
    while True:
    # for i in range(1):
        V_new = np.zeros(len(states))
        delta = 0
        for s, _ in enumerate(states):
            v_fn = 0
            action_probs = policy[s]
            for a, _ in enumerate(GATES):
                p_trans = transitions[a][s]
                p_next_states = np.nonzero(transitions[a][s])[0]
                for next_s in p_next_states:
                    v_fn += action_probs[a] * p_trans[next_s] * (R(states[s], a) + discount_factor * V_old[next_s])
            delta = max(delta, abs(v_fn - V_old[s]))
            V_new[s] = v_fn
        V_old = V_new
        if(delta < epsilon):
            print('converged')
            break
    # since technically the entire north/south pole is one state, copy (0, 0) and (k-1, k-1) over
    # won't ever be used, but it is needed for the visualization

    for i in range(k):
        ind1 = states.index((0,0,i))
        ind2 = states.index((k-1, k-1, i))
        for j in range(1, len(phis)):
            V_old[ind1 + j*k] = V_old[ind1]
            V_old[ind2 - j*k] = V_old[ind2]
    return np.array(V_old)

In [19]:
def policy_improvement(policy_eval_fn=policy_eval, discount_factor=0.8):      
    def one_step_lookahead(s, V_old):
        actions = np.zeros(len(GATES))
        for a in range(len(GATES)):
            v_fn = 0
            p_trans = transitions[a][s]
            p_next_states = np.nonzero(transitions[a][s])[0]
            for next_s in p_next_states:
                v_fn += p_trans[next_s] * (R(states[s], a) + discount_factor * V_old[next_s])
            actions[a] = v_fn
        return actions
    policy = np.ones([len(states), len(GATES)]) / len(GATES)
    actions_values = np.zeros(len(GATES))
    
    while True:
        value_fn = policy_eval_fn(policy)
        policy_stable = True
        for s in range(len(states)):
            actions_values = one_step_lookahead(s, value_fn)
            best_action = np.argmax(actions_values)
            chosen_action = np.argmax(policy[s])
            if(best_action != chosen_action):
                policy_stable = False
            policy[s] = np.eye(len(GATES))[best_action]
        
        if(policy_stable):
            return policy, value_fn

In [32]:
policy, v = policy_improvement(policy_eval)

converged
converged
converged
converged
converged
converged
converged


In [237]:
n = 10**4
goal = generate_target_state(n=n)
goal_reg = dm_to_bloch_reg(goal)

with np.load(f'./results/{t1}_{t2}_{gate_time}/policy_{n}_{k}_{t1}_{t2}_{gate_time}.npz') as data:
    policy = data['arr_0']
with np.load(f'./results/{t1}_{t2}_{gate_time}/v_{n}_{k}_{t1}_{t2}_{gate_time}.npz') as data:
    v = data['arr_0']

prev_seq = [
    [0,1,0,1,0,1,0,1,1],
    [0,1,1,1,0,1,0,1,1,1],
    [0,1,0],
    [0,1,0],
    [0,1,0,1,1,1,1,0],
    [0,1,1,1,0,1,0,1,0,1,0,1,1,1,1,1,0],
    [2],
    [2],
    [0,1,0,1,0,1,0,1,1,1,0]
]

In [242]:
optimal_programs = []
for i in range(k):
    converged = False
    while not converged:
        s = random_state_in_reg((0, 0, k-1))
        prog = []
        counter = 0
        while counter < 10:
            action = np.argmax(policy[states.index(dm_to_bloch_reg(s))])
            next_s = apply_operator(s, GATES[action])
            prog.append(action)
            next_s = random_state_in_reg(dm_to_bloch_reg(next_s))
            s = next_s
            counter += 1
            reg = dm_to_bloch_reg(s)
            if (reg[0] == goal_reg[0] and reg[1] == goal_reg[1]):
                print('converged')
                converged = True
                break
        
    optimal_programs.append(prog)

converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged


In [243]:
print(goal_reg)
for prog in optimal_programs:
    psi = np.matrix([1,0])
    rho = np.matrix(np.outer(psi, psi.H))

    for g in prog:
        rho = apply_operator(rho, GATES[g])
    print(prog, dm_fidelity(goal, rho), dm_to_bloch_reg(rho))

(4, 24, 15)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 0, 1, 1, 1, 1, 0, 1, 0] (0.8195179083072464+1.1687313370438272e-09j) (4, 26, 10)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.7117864927293851-9.54096213894399e-09j) (4, 25, 6)
[0, 1, 1, 1, 1, 0, 1, 0] (0.711786

In [225]:
alt_progs = []
alt_progs.append(prev_seq[int(np.log10(n))-2])

for prog in alt_progs:
    psi = np.matrix([1,0])
    rho = np.matrix(np.outer(psi, psi.H))

    for g in prog:
        rho = apply_operator(rho, GATES[g])
    print(prog, dm_fidelity(goal, rho), dm_to_bloch_reg(rho))

[0, 1, 0] (0.8433699366855134-5.997690755443918e-09j) (4, 27, 11)


In [118]:
goal1 = generate_target_state(10**7)
goal2 = generate_target_state(10**10)
print(dm_fidelity(goal1, goal2))

(0.9967441557736434+4.1058395846576135e-09j)


In [124]:
goal1 = generate_target_state(10**3)
goal2 = generate_target_state(10**4)
print(dm_fidelity(goal1, goal2))

(0.9997293830774311+6.685997845237901e-10j)
